In [15]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle
DATA_DIR = '/mnt/chicm/data/open-images/relation'
DETECT_DATA_DIR = '/mnt/chicm/data/open-images/detect'

In [3]:
!ls -lh

total 268M
-rw-rw-r-- 1 core core 1.3M Jul 21 18:58 eda.ipynb
-rw-rw-r-- 1 core core 266M Jul 22 16:57 sub_detect_0722_1.csv
-rw-rw-r-- 1 core core 1.1M Jul 22 16:54 submit-detect.ipynb
-rw-r--r-- 1 core core  59K Jul 22 15:17 submit.ipynb


In [6]:
df_box = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-train-vrd-bbox.csv'))
df_box.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax,IsGroupOf
0,780d8a470d144fa0,/m/04yx4,0.000000,0.527143,0.029979,0.997859,0
1,780d8a470d144fa0,/m/04yx4,0.291429,0.687143,0.128480,0.997859,0
2,780d8a470d144fa0,/m/04yx4,0.590000,0.965714,0.164882,0.997859,0
3,780db0a7d07a4a20,/m/026t6,0.000000,1.000000,0.345625,0.990000,-1
4,780db0a7d07a4a20,/m/026t6,0.271667,0.626667,0.803125,0.999375,0


In [99]:
df_box.LabelName.value_counts()

/m/04yx4      1418594
/m/03bt1vf     767337
/m/0k4j        248075
/m/05r655      197155
/m/01mzpv      132483
/m/01bl7v       87555
/m/04bcr3       85691
/m/04dr76w      40188
/m/0199g        40161
/m/0bt9lr       28675
/m/0hg7b        27272
/m/0342h        25896
/m/026t6        24818
/m/01yrx        15183
/m/04_sv        13382
/m/03k3r        13368
/m/09tvcd       12934
/m/01y9k5       11693
/m/01599         9565
/m/0h2r6         7720
/m/0cvnqh        7229
/m/0dv5r         6404
/m/050k8         6365
/m/02p5f1q       5327
/m/078n6m        5314
/m/01226z        5097
/m/0pg52         4199
/m/03ssj5        3563
/m/071p9         3505
/m/0bwd_0j       3272
/m/08pbxl        3026
/m/07y_7         2828
/m/019w40        2594
/m/080hkjn       2495
/m/02jvh9        2272
/m/0cmx8         1709
/m/0dt3t         1687
/m/02hj4         1532
/m/03m3pdh       1501
/m/078jl         1378
/m/05r5c         1374
/m/01940j        1216
/m/0h8my_4        985
/m/06__v          944
/m/04ctx          850
/m/05_5p_0

In [20]:
print(df_box.shape, len(df_box.ImageID.unique()), len(df_box.LabelName.unique()))

(3290070, 7) 790478 57


In [7]:
df_vrd = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-train-vrd.csv'))
df_vrd.head()

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.00500,0.033125,0.580000,0.627778,0.00500,0.033125,0.580000,0.627778,is
1,82d16a22f703df5c,/m/04dr76w,/m/02gy9n,0.61200,0.735000,0.418919,0.950450,0.61200,0.735000,0.418919,0.950450,is
2,b54d41beabcfd900,/m/01mzpv,/m/083vt,0.37250,0.399375,0.706413,0.778557,0.37250,0.399375,0.706413,0.778557,is
3,4b6a08cc110d7275,/m/01mzpv,/m/01y9k5,0.17125,0.255625,0.557500,0.749167,0.20750,0.683125,0.611667,0.999167,at
4,0144cfbb726f4c72,/m/01mzpv,/m/04bcr3,0.85500,0.950000,0.561667,0.609167,0.82875,0.999375,0.568333,0.673333,at


In [8]:
print(df_vrd.shape, len(df_vrd.ImageID.unique()), len(df_vrd.LabelName1.unique()),\
      len(df_vrd.LabelName2.unique()), len(df_vrd.RelationshipLabel.unique()))

(374768, 12) 100522 32 58 10


In [8]:
df_vrd.loc[df_vrd.ImageID=='fe58ec1b06db2bb7']

,ImageID,LabelName1,LabelName2,XMin1,XMax1,YMin1,YMax1,XMin2,XMax2,YMin2,YMax2,RelationshipLabel
0,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.005000,0.033125,0.580000,0.627778,0.005000,0.033125,0.580000,0.627778,is
77712,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.071875,0.121875,0.597778,0.680000,0.071875,0.121875,0.597778,0.680000,is
171028,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.124375,0.217500,0.612222,0.712222,0.124375,0.217500,0.612222,0.712222,is
230777,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.185000,0.308125,0.593333,0.768889,0.185000,0.308125,0.593333,0.768889,is
284575,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.513750,0.903125,0.680000,0.998889,0.513750,0.903125,0.680000,0.998889,is
362652,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.308750,0.517500,0.632222,0.864444,0.308750,0.517500,0.632222,0.864444,is
370350,fe58ec1b06db2bb7,/m/04bcr3,/m/083vt,0.038125,0.071875,0.582222,0.662222,0.038125,0.071875,0.582222,0.662222,is


In [9]:
df_box.loc[df_box.ImageID=='fe58ec1b06db2bb7']

,ImageID,LabelName,XMin,XMax,YMin,YMax,IsGroupOf
2055608,fe58ec1b06db2bb7,/m/04bcr3,0.185000,0.308125,0.593333,0.768889,0
2055609,fe58ec1b06db2bb7,/m/04bcr3,0.005000,0.033125,0.580000,0.627778,0
2055610,fe58ec1b06db2bb7,/m/04bcr3,0.038125,0.071875,0.582222,0.662222,0
2055611,fe58ec1b06db2bb7,/m/04bcr3,0.071875,0.121875,0.597778,0.680000,0
2055612,fe58ec1b06db2bb7,/m/04bcr3,0.124375,0.217500,0.612222,0.712222,0
2055613,fe58ec1b06db2bb7,/m/04bcr3,0.308750,0.517500,0.632222,0.864444,0
2055614,fe58ec1b06db2bb7,/m/04bcr3,0.513750,0.903125,0.680000,0.998889,0


In [22]:
df_vrd.RelationshipLabel.value_counts()

is                194142
at                111493
on                 31604
holds              20986
plays               8932
interacts_with      3756
inside_of           2392
wears                836
hits                 593
under                 34
Name: RelationshipLabel, dtype: int64

In [25]:
df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName2.value_counts()

/m/083vt     117165
/m/05z87      36140
/m/02gy9n     20644
/m/04lbp      14411
/m/0dnr7       5782
Name: LabelName2, dtype: int64

In [43]:
df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName1.value_counts()

/m/01mzpv     70456
/m/04bcr3     48975
/m/04dr76w    27270
/m/0342h      13444
/m/01y9k5      7472
/m/0cvnqh      5089
/m/080hkjn     3820
/m/078n6m      3696
/m/026t6       2906
/m/07y_7       2550
/m/03m3pdh     2129
/m/03ssj5      1370
/m/01940j      1276
/m/05r5c       1182
/m/01s55n       867
/m/01_5g        574
/m/0584n8       290
/m/071p9        204
/m/02p5f1q      195
/m/0cmx8        151
/m/02jvh9       141
/m/0dt3t         55
/m/04ctx         30
Name: LabelName1, dtype: int64

In [9]:
classes_is = df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName1.unique().tolist()

In [10]:
classes_is

['/m/04bcr3',
 '/m/04dr76w',
 '/m/01mzpv',
 '/m/078n6m',
 '/m/0342h',
 '/m/03m3pdh',
 '/m/03ssj5',
 '/m/01_5g',
 '/m/01y9k5',
 '/m/0cvnqh',
 '/m/07y_7',
 '/m/071p9',
 '/m/05r5c',
 '/m/01940j',
 '/m/01s55n',
 '/m/080hkjn',
 '/m/026t6',
 '/m/02p5f1q',
 '/m/0cmx8',
 '/m/0dt3t',
 '/m/0584n8',
 '/m/04ctx',
 '/m/02jvh9']

In [11]:
df_classes = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-classes-vrd.csv'), names=['classes', 'desc'])
df_classes.head()

,classes,desc
0,/m/05r5c,Piano
1,/m/01599,Beer
2,/m/01_5g,Chopsticks
3,/m/01yrx,Cat
4,/m/04_sv,Motorcycle


In [16]:
df_detect_classes = pd.read_csv(os.path.join(DETECT_DATA_DIR, 'challenge-2019-classes-description-500.csv'), names=['classes', 'desc2'])
df_detect_classes.head()

,classes,desc2
0,/m/061hd_,Infant bed
1,/m/06m11,Rose
2,/m/03120,Flag
3,/m/01kb5b,Flashlight
4,/m/0120dh,Sea turtle


In [17]:
df_join = df_classes.set_index('classes').join(df_detect_classes.set_index('classes'), on='classes')
df_join.head(50)

,desc,desc2
classes,,
/m/05r5c,Piano,Piano
/m/01599,Beer,Beer
/m/01_5g,Chopsticks,Chopsticks
/m/01yrx,Cat,Cat
/m/04_sv,Motorcycle,Motorcycle
/m/0h8my_4,Tennis racket,Tennis racket
/m/01226z,Football,Football
/m/050k8,Mobile phone,Mobile phone
/m/0l14j_,Flute,Flute


In [18]:
df_join.shape

(57, 2)

In [19]:
!ls -lh

total 268M
-rw-rw-r-- 1 core core  1.3M Jul 21 18:58 eda.ipynb
-rw-rw-r-- 1 core core  266M Jul 22 16:57 sub_detect_0722_1.csv
-rw-rw-r-- 1 core core 1011K Jul 22 16:59 submit-detect.ipynb
-rw-r--r-- 1 core core   52K Jul 22 17:02 submit.ipynb


In [55]:
df_det = pd.read_csv('sub_detect_0722_1.csv')
df_det.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.7549744 0.0000000 0.1213381 0.43152...
1,763c72626bf24534,/m/04yx4 0.9219856 0.0016186 0.1897768 0.25732...
2,7540c162180973b1,/m/04yx4 0.6974359 0.5084532 0.0000000 0.60996...
3,c4e522dce52cb02f,/m/04yx4 0.8264070 0.0000000 0.1761055 0.21462...
4,cafcd9496c808085,/m/0k4j 0.0046642 0.8008922 0.0305311 0.999023...


In [56]:
def get_det(pred_str):
    dets = []
    det = []
    for i, e in enumerate(pred_str.split(' ')):
        if i % 6 == 0:
            det = []
        det.append(e)
        if (i+1) % 6 == 0:
            if det[0] in set(classes_is) and float(det[1]) > 0.1:
                dets.append(det)
            
    return dets

In [57]:
df_det['dets'] = df_det.PredictionString.map(lambda x: get_det(str(x)))
df_det.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,/m/04yx4 0.7549744 0.0000000 0.1213381 0.43152...,"[[/m/01mzpv, 0.4466539, 0.0203321, 0.6938528, ..."
1,763c72626bf24534,/m/04yx4 0.9219856 0.0016186 0.1897768 0.25732...,"[[/m/04bcr3, 0.8843107, 0.6863727, 0.8291983, ..."
2,7540c162180973b1,/m/04yx4 0.6974359 0.5084532 0.0000000 0.60996...,"[[/m/01mzpv, 0.2206312, 0.3038167, 0.6631562, ..."
3,c4e522dce52cb02f,/m/04yx4 0.8264070 0.0000000 0.1761055 0.21462...,"[[/m/01mzpv, 0.1240795, 0.4363872, 0.6185770, ..."
4,cafcd9496c808085,/m/0k4j 0.0046642 0.8008922 0.0305311 0.999023...,"[[/m/04bcr3, 0.2282194, 0.0039188, 0.0031391, ..."


In [58]:
len(df_det.iloc[0].dets)

7

In [54]:
df_vrd.loc[df_vrd.RelationshipLabel=='is'].LabelName2.unique()

array(['/m/083vt', '/m/02gy9n', '/m/05z87', '/m/04lbp', '/m/0dnr7'],
      dtype=object)

In [59]:
def get_rel_pred(dets):
    preds = []
    for det in dets:
        for label2 in ['/m/083vt', '/m/02gy9n', '/m/05z87', '/m/04lbp', '/m/0dnr7']:
            preds.append(det[1]) # conf
            preds.append(det[0]) # label1
            preds.append(det[2])
            preds.append(det[3])
            preds.append(det[4])
            preds.append(det[5])

            preds.append(label2) #label2
            preds.append(det[2])
            preds.append(det[3])
            preds.append(det[4])
            preds.append(det[5])
            preds.append('is')
    preds = [str(x) for x in preds]
    return ' '.join(preds)

In [60]:
df_det.PredictionString = df_det.dets.map(lambda x: get_rel_pred(x))

In [61]:
df_det.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,0.4466539 /m/01mzpv 0.0203321 0.6938528 0.4087...,"[[/m/01mzpv, 0.4466539, 0.0203321, 0.6938528, ..."
1,763c72626bf24534,0.8843107 /m/04bcr3 0.6863727 0.8291983 0.9988...,"[[/m/04bcr3, 0.8843107, 0.6863727, 0.8291983, ..."
2,7540c162180973b1,0.2206312 /m/01mzpv 0.3038167 0.6631562 0.5975...,"[[/m/01mzpv, 0.2206312, 0.3038167, 0.6631562, ..."
3,c4e522dce52cb02f,0.1240795 /m/01mzpv 0.4363872 0.6185770 0.6537...,"[[/m/01mzpv, 0.1240795, 0.4363872, 0.6185770, ..."
4,cafcd9496c808085,0.2282194 /m/04bcr3 0.0039188 0.0031391 0.9755...,"[[/m/04bcr3, 0.2282194, 0.0039188, 0.0031391, ..."


In [62]:
df_det.to_csv('sub_is_th0.1_all_label2_0723.csv', columns=['ImageId', 'PredictionString'], index=False)

In [98]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f sub2_is_th0.1.csv -m "th0.1 submit"

100%|██████████████████████████████████████| 25.7M/25.7M [00:02<00:00, 9.24MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship

In [52]:
!ls -lh

total 439M
-rw-rw-r-- 1 core core  1.3M Jul 21 18:58 eda.ipynb
-rw-rw-r-- 1 core core  266M Jul 22 16:57 sub_detect_0722_1.csv
-rw-rw-r-- 1 core core  138M Jul 22 17:06 sub_is_th0.01_0723.csv
-rw-rw-r-- 1 core core   33M Jul 22 17:08 sub_is_th0.1_0723.csv
-rw-rw-r-- 1 core core 1011K Jul 22 16:59 submit-detect.ipynb
-rw-r--r-- 1 core core   56K Jul 22 17:08 submit.ipynb
